In [3]:
from typing import List, Optional, Tuple
import math
import random

# ============================
# Pagalbinės: spausdinimas ir normos
# ============================
def print_matrix(M: List[List[float]], title: str = "", fmt: str = "{:11.6g}"):
    if title:
        print(f"\n{title}\n" + "-" * len(title))
    for row in M:
        print(" ".join(fmt.format(v) for v in row))
    print()

def print_vector(v: List[float], title: str = "", fmt: str = "{:11.6g}"):
    if title:
        print(f"\n{title}\n" + "-" * len(title))
    print(" ".join(fmt.format(x) for x in v))
    print()

def deep_copy_matrix(A: List[List[float]]) -> List[List[float]]:
    return [row[:] for row in A]

def matvec(A: List[List[float]], x: List[float]) -> List[float]:
    return [sum(A[i][j] * x[j] for j in range(len(x))) for i in range(len(A))]

def vec_add(a: List[float], b: List[float]) -> List[float]:
    return [a[i] + b[i] for i in range(len(a))]

def vec_sub(a: List[float], b: List[float]) -> List[float]:
    return [a[i] - b[i] for i in range(len(a))]

def vec_dot(a: List[float], b: List[float]) -> float:
    return sum(a[i] * b[i] for i in range(len(a)))

def norm2_vec(v: List[float]) -> float:
    return math.sqrt(sum(x*x for x in v))

def transpose(A: List[List[float]]) -> List[List[float]]:
    return [list(row) for row in zip(*A)]

def eye(n: int) -> List[List[float]]:
    return [[1.0 if i==j else 0.0 for j in range(n)] for i in range(n)]

def check_solution(A: List[List[float]], x: Optional[List[float]], b: List[float], tag: str):
    if x is None:
        print(f"[{tag}] sprendinio nėra ką tikrinti.")
        return
    Ax = matvec(A, x)
    r  = vec_sub(b, Ax)
    print_vector(Ax, f"{tag}: A*x")
    print_vector(r,  f"{tag}: liekana r = b - A*x")
    print(f"{tag}: ||x||2 = {norm2_vec(x):.12g},  ||r||2 = {norm2_vec(r):.12g}\n")

# ============================
# Skaidrių (SMA_04) perstatymas į formulę:
# Atld = diag(1/diag(A))*A - diag(alpha),  btld = diag(1/diag(A))*b
# PI:  x^{k+1} = (btld - Atld * x^k) ./ alpha
# GZ:  x^{k+1}_i = (btld_i - Atld_i * x^{k+1}) / alpha_i  (naudojant naujausias reikšmes)
# ============================
def build_Atld_btld(A: List[List[float]], b: List[float], alpha: List[float], verbose: bool=True):
    n = len(A)
    Dinv = [1.0/A[i][i] if abs(A[i][i])>1e-15 else float('inf') for i in range(n)]
    if any(math.isinf(x) for x in Dinv):
        raise ZeroDivisionError("Diag(A) turi nulį — perstatyk lygtis/kintamuosius (skaidrės p.10).")
    # btld = D^{-1} b
    btld = [Dinv[i]*b[i] for i in range(n)]
    # Atld = D^{-1} A - diag(alpha)
    Atld = [[Dinv[i]*A[i][j] for j in range(n)] for i in range(n)]
    for i in range(n):
        Atld[i][i] -= alpha[i]
    if verbose:
        print_matrix(A, "Pradinė A")
        print_vector(b, "Pradinė b")
        print_vector([A[i][i] for i in range(n)], "Įstrižainė(A)")
        print_vector(alpha, "α (metodo parametrai)")
        print_matrix(Atld, "A~ = D^{-1}A - diag(α)")
        print_vector(btld, "b~ = D^{-1} b")
    return Atld, btld

# ============================
# 1) Paprastųjų iteracijų (PI) algoritmas
# ============================
def simple_iterations(A: List[List[float]], b: List[float],
                      alpha: Optional[List[float]] = None,
                      x0: Optional[List[float]] = None,
                      eps: float = 1e-12, nitmax: int = 1000, verbose: bool=True) -> Optional[List[float]]:
    n = len(A)
    if alpha is None: alpha = [1.0]*n
    if x0 is None:    x0    = [0.0]*n

    Atld, btld = build_Atld_btld(A, b, alpha, verbose=verbose)
    x = x0[:]
    print_vector(x, "PI: pradinis artinys x^(0)")

    for it in range(1, nitmax+1):
        # x1 = (btld - Atld * x) ./ alpha
        Ax = matvec(Atld, x)
        rhs = [btld[i] - Ax[i] for i in range(n)]
        x1  = [rhs[i] / alpha[i] for i in range(n)]
        # tikslumas = ||x1 - x||/(||x1||+||x||)
        prec = norm2_vec(vec_sub(x1, x)) / max(1e-30, (norm2_vec(x1) + norm2_vec(x)))
        if verbose:
            print(f"[PI] it={it:4d}  prec={prec:.3e}")
            if it <= 5 or it % 10 == 0:
                print_vector(x1, f"[PI] x^{it}")
        x = x1
        if prec < eps:
            print(f"[PI] Konvergavo per {it} iteracijų (eps={eps}).")
            break
    else:
        print("[PI] DĖMESIO: pasiektas nitmax be konvergencijos.")

    print_vector(x, "PI: gautas x")
    check_solution(A, x, b, "PI")
    return x

# ============================
# 2) Gauso–Zeidelio (GZ) algoritmas
# ============================
def gauss_seidel(A: List[List[float]], b: List[float],
                 alpha: Optional[List[float]] = None,
                 x0: Optional[List[float]] = None,
                 eps: float = 1e-12, nitmax: int = 1000, verbose: bool=True) -> Optional[List[float]]:
    n = len(A)
    if alpha is None: alpha = [1.0]*n
    if x0 is None:    x0    = [0.0]*n

    Atld, btld = build_Atld_btld(A, b, alpha, verbose=verbose)
    x  = x0[:]
    x1 = x0[:]
    print_vector(x, "GZ: pradinis artinys x^(0)")

    for it in range(1, nitmax+1):
        # eilutė po eilutės: x1[i] = (btld[i] - Atld[i,:] @ x1) / alpha[i]
        for i in range(n):
            row_dot = sum(Atld[i][j]*x1[j] for j in range(n))  # NAUDOJAM NAUJAS x1 reikšmes
            x1[i] = (btld[i] - row_dot) / alpha[i]
        prec = norm2_vec(vec_sub(x1, x)) / max(1e-30, (norm2_vec(x1) + norm2_vec(x)))
        if verbose:
            print(f"[GZ] it={it:4d}  prec={prec:.3e}")
            if it <= 5 or it % 10 == 0:
                print_vector(x1, f"[GZ] x^{it}")
        x = x1[:]
        if prec < eps:
            print(f"[GZ] Konvergavo per {it} iteracijų (eps={eps}).")
            break
    else:
        print("[GZ] DĖMESIO: pasiektas nitmax be konvergencijos.")

    print_vector(x1, "GZ: gautas x")
    check_solution(A, x1, b, "GZ")
    return x1

# ============================
# 3) Matricos sąlygotumo skaičius κ2(A)
# ============================
# -- Nedidelė LU su daliniu perstatymu, skirta fallback metodui
def lu_decomposition(A_in: List[List[float]]):
    A = deep_copy_matrix(A_in)
    n = len(A)
    U = deep_copy_matrix(A)
    L = [[0.0]*n for _ in range(n)]
    P = [[1.0 if i==j else 0.0 for j in range(n)] for i in range(n)]
    for k in range(n):
        L[k][k] = 1.0
    for k in range(n-1):
        pivot = max(range(k, n), key=lambda i: abs(U[i][k]))
        if abs(U[pivot][k]) < 1e-15:
            raise ZeroDivisionError("LU: singuliari matrica (vedantysis≈0).")
        if pivot != k:
            U[k], U[pivot] = U[pivot], U[k]
            P[k], P[pivot] = P[pivot], P[k]
            for j in range(k):
                L[k][j], L[pivot][j] = L[pivot][j], L[k][j]
        for i in range(k+1, n):
            L[i][k] = U[i][k] / U[k][k]
            U[i][k] = 0.0
            for j in range(k+1, n):
                U[i][j] -= L[i][k] * U[k][j]
    return P, L, U

def forward_substitution(L: List[List[float]], b: List[float]) -> List[float]:
    n = len(L)
    y = [0.0]*n
    for i in range(n):
        s = sum(L[i][j]*y[j] for j in range(i))
        y[i] = (b[i]-s)/L[i][i]
    return y

def back_substitution(U: List[List[float]], y: List[float]) -> List[float]:
    n = len(U)
    x = [0.0]*n
    for i in range(n-1, -1, -1):
        s = sum(U[i][j]*x[j] for j in range(i+1, n))
        x[i] = (y[i]-s)/U[i][i]
    return x

def apply_P(P: List[List[float]], b: List[float]) -> List[float]:
    return [sum(P[i][j]*b[j] for j in range(len(b))) for i in range(len(b))]

def solve_via_LU(P: List[List[float]], L: List[List[float]], U: List[List[float]], b: List[float]) -> List[float]:
    Pb = apply_P(P, b)
    y  = forward_substitution(L, Pb)
    x  = back_substitution(U, y)
    return x

def spectral_norm_estimate(A: List[List[float]], iters: int = 50) -> float:
    """ ||A||_2 ≈ sqrt(lambda_max(A^T A)) — paprastas galios metodas. """
    n = len(A)
    AT = transpose(A)
    x = [random.random() for _ in range(n)]
    for _ in range(iters):
        y  = matvec(AT, matvec(A, x))  # (A^T A) x
        ny = norm2_vec(y)
        if ny < 1e-30:
            return 0.0
        x  = [yy/ny for yy in y]
    # spindulio įvertis
    y = matvec(AT, matvec(A, x))
    lam = vec_dot(x, y)  # Rayleigh
    return math.sqrt(max(0.0, lam))

def inv_spectral_norm_estimate(A: List[List[float]], iters: int = 50) -> float:
    """ ||A^{-1}||_2 ≈ sqrt(lambda_max((A^{-1})^T A^{-1})).
        Įgyvendiname kaip kartotinės užduotys su tais pačiais LU:
        (A^{-1})^T A^{-1} x = z  ⇔ iš pradžių u = A^{-1} x, tada z = (A^{-1})^T u.
        (A^{-1})^T u gaunam spręsdami A^T z = u.
    """
    n = len(A)
    # Vienkartinė LU A ir A^T
    P, L, U   = lu_decomposition(A)
    AT        = transpose(A)
    PT, LT, UT = lu_decomposition(AT)

    x = [random.random() for _ in range(n)]
    nx = norm2_vec(x)
    if nx < 1e-30:
        x = [1.0] + [0.0]*(n-1)
    else:
        x = [xi/nx for xi in x]

    for _ in range(iters):
        # u = A^{-1} x
        u = solve_via_LU(P, L, U, x)
        # z = (A^{-1})^T u ⇒ spręsti A^T z = u
        z = solve_via_LU(PT, LT, UT, u)
        nz = norm2_vec(z)
        if nz < 1e-30:
            return 0.0
        x = [zi/nz for zi in z]
    # Rayleigh įvertis: x^T ((A^{-1})^T A^{-1}) x = ||A^{-1} x||^2
    u = solve_via_LU(P, L, U, x)
    lam = vec_dot(u, u)   # = ||A^{-1} x||_2^2
    return math.sqrt(max(0.0, lam))

def condition_number(A: List[List[float]], prefer_numpy: bool = True, verbose: bool=True) -> float:
    if prefer_numpy:
        try:
            import numpy as np
            Anp = np.array(A, dtype=float)
            k2  = float(np.linalg.cond(Anp, 2))
            if verbose:
                print(f"cond_2(A) (NumPy) = {k2:.12g}")
            return k2
        except Exception:
            if verbose:
                print("NumPy nepasiekiama – skaičiuoju įverčius be bibliotekų.")

    nA   = spectral_norm_estimate(A, iters=60)
    nAinv= inv_spectral_norm_estimate(A, iters=60)
    k2   = nA * nAinv
    if verbose:
        print(f"||A||_2 ≈ {nA:.12g},  ||A^-1||_2 ≈ {nAinv:.12g}")
        print(f"cond_2(A) ≈ {k2:.12g}  (įverčiai)")
    return k2

# ============================
# MAIN — tik paleidimų „stub’ai“ (užkomentuoti)
# ============================
def main():
    # PAVYZDYS
    A = [
        [1.0,  1.0,  1.0,  1.0],
        [1.0, -5.0, -1.0,  1.0],
        [2.0,  1.0, -10.0, 2.0],
        [3.0,  1.0,  2.0, -10.0],
    ]
    b = [2.0, 0.0, 9.0, -7.0]

    # --- Paprastųjų iteracijų metodas ---
    # x_pi = simple_iterations(A, b, alpha=[1.8,1.8,1.8,1.8], x0=[0,0,0,0], eps=1e-12, nitmax=500, verbose=True)

    # --- Gauso–Zeidelio metodas ---
    # x_gz = gauss_seidel(A, b, alpha=[1.1,1.1,1.1,1.1], x0=[0,0,0,0], eps=1e-12, nitmax=500, verbose=True)

    # --- Matricos sąlygotumo skaičius ---
    # kappa = condition_number(A, prefer_numpy=True, verbose=True)

    pass

main()


cond_2(A) (NumPy) = 7.36826712046
